# 트랜스포머야!
- 트랜스포머지.
- 일단 모델 설계부터 먼저 해볼까?

In [2]:
import torch

import torch.nn as nn

## 데이터 대충 가져오기

In [12]:
from datasets import load_dataset

dataset = load_dataset("imdb")
train_dataset = dataset['train']
test_dataset = dataset['test']

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 'unsupervised' 데이터셋 제거
del dataset['unsupervised']

# 'train'과 'test' 데이터셋 크기 조정
dataset['train'] = dataset['train'].select(range(10000))
dataset['test'] = dataset['test'].select(range(10000))


In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})

In [14]:
def tokenize_function(examples):
    tokenized = tokenizer(examples['text'], padding="max_length", truncation=True)
    
    # 특정 필드만 float로 변환 (예: 'input_ids')
    if 'input_ids' in tokenized:
        tokenized['input_ids'] = [torch.tensor(ids, dtype=torch.float) for ids in tokenized['input_ids']]
    
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [15]:
from torch.utils.data import DataLoader

train_dataset = tokenized_datasets["train"].remove_columns(["text"])
train_dataset = train_dataset.rename_column("label", "labels")
train_dataset.set_format("torch")

test_dataset = tokenized_datasets["test"].remove_columns(["text"])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format("torch")

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

## 일단 포지셔널 인코딩부터 만들자

In [6]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_seq_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.seq_len = max_seq_len
        self.d_model = d_model
        self.positional_encoding = self.get_pos_encoding(max_seq_len, d_model)

    def get_angles(self, position, i, d_model):
        angles = position / torch.pow(10000, (2 * (i // 2)) / d_model)
        return angles

    def get_pos_encoding(self, max_seq_len, d_model):
        angle_tensor = self.get_angles(
            position=torch.arange(max_seq_len).unsqueeze(1),
            i=torch.arange(d_model).unsqueeze(0),
            d_model=torch.tensor(d_model, dtype=torch.float32)
        )
        positional_encoding = torch.zeros(max_seq_len, d_model, dtype=torch.float32)
        positional_encoding[:, 0::2] = torch.sin(angle_tensor[:, 0::2])
        positional_encoding[:, 1::2] = torch.cos(angle_tensor[:, 1::2])
        return positional_encoding

    def forward(self, x):
        seq_len = x.size(1)
        return x + self.positional_encoding[:seq_len, :].unsqueeze(0).to(x.device)




In [42]:
### pe 실험
max_seq_len = 512
d_model = 512

x = torch.linspace(0, 1, max_seq_len).unsqueeze(0)
pe =  PositionalEncoding(max_seq_len=max_seq_len, d_model=d_model)
encoded_x = pe(x)

# encoded_x =  x +  pe[:x.size(0), :]
print(encoded_x.shape)



torch.Size([1, 512, 512])


## encoder block

In [62]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, use_encoder_output = False):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.use_encoder_output = use_encoder_output
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.layer_norm = nn.LayerNorm(d_model)
    
    def split_heads(self, x):
        batch_size, seq_length, _ = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
    
    def forward(self, x, mask, encoder_x = None):
        batch_size, seq_length, _ = x.size()
        
        # Linear projections
        q = self.W_q(x)
        if self.use_encoder_output:
            k = self.W_k(encoder_x)
            v = self.W_v(encoder_x)
        else:
            k = self.W_k(x)
            v = self.W_v(x)
        
        # Split heads
        q = self.split_heads(q)
        k = self.split_heads(k)
        v = self.split_heads(v)
        ## 이때 각각의 차원은 (batch_size, num_heads, seq_length, d_k)

        
        # Scaled dot-product attention
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))

        ## k.transpose 의 차원 (batch_size, num_heads, d_k, seq_length)
        ## attention_scores의 차원 (batch_size, num_heads, seq_length, seq_length)
        ## mask의 차원은 (batch_size, 1, seq_length, seq_length)
        if mask is not None:
            attention_scores += mask * (-1e9)
        
        attention_probs = torch.softmax(attention_scores, dim=-1)
        ### 이러면 (batch_size, num_heads, seq_length, seq_length)가 되고
        context = torch.matmul(attention_probs, v)
        ### 여기서 (batch_size, num_heads, seq_length, d_k)가 된다.
        
        # Combine heads
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        ## 여기서 x와 같은 차원(batch_size, seq_length, d_model)이 된다.
        output = self.W_o(context)
        
        # return output
        return self.layer_norm(x + output) ## ResidualConnection을 위해 더해주기, # LayerNormalization 해주기


In [45]:
class FFNN(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FFNN, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.linear1 = nn.Linear(d_model, d_ff, bias=True)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(d_ff, d_model, bias=True)

        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        output = self.linear1(x)
        output = self.relu(x)
        output = self.linear2(x)

        return self.layer_norm(x + output)


In [ ]:
import torch.nn as nn

class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, num_heads):
        super(EncoderLayer, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, d_model=d_model)
        self.ffnn = FFNN(d_model=d_model, d_ff=d_ff)

    def forward(self, x, mask):
        x = self.att(x, mask)
        x = self.ffnn(x)
        return x





## decoder block

In [41]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, num_heads):
        super(DecoderLayer, self).__init__()
        self.att1 = MultiHeadAttention(num_heads=num_heads, d_model=d_model)
        self.att2 = MultiHeadAttention(num_heads=num_heads, d_model=d_model, use_encoder_output=True)
        self.ffnn = FFNN(d_model=d_model, d_ff=d_ff)

    def forward(self, x, mask, encoded_x):
        if mask is None:
            n = x.size(1)
            look_ahead_mask = torch.triu(torch.ones(n, n), diagonal=1)  
        else:
            look_ahead_mask = torch.logical_or(mask, torch.triu(torch.ones(n, n), diagonal=1))
        x = self.att1(x, look_ahead_mask)
        x = self.att2(x, mask, encoded_x = encoded_x)
        x = self.ffnn(x)
        return x

In [40]:
class MyTransformer(nn.Module):
    def __init__(self, seq_len, d_model, d_ff, num_heads, num_layers=6):
        super(MyTransformer, self).__init__()
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, d_ff, num_heads) for _ in range(num_layers)
        ])
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, d_ff, num_heads) for _ in range(num_layers)
        ])
        self.pe = PositionalEncoding(seq_len=seq_len, d_model=d_model)

    def forward(self, src, tgt, src_mask, tgt_mask):
        src = self.pe(src)
        tgt = self.pe(tgt)

        # Encoder
        encoded_src = src
        for layer in self.encoder_layers:
            encoded_src = layer(encoded_src, src_mask)

        # Decoder
        for layer in self.decoder_layers:
            tgt = layer(tgt, tgt_mask, encoded_x=encoded_src)
        
        return tgt


tensor([[0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]])